In [1]:
from numpy.random import seed
seed(1)

# Data Pre-Processing

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [3]:
import numpy as np
import pandas as pd

In [85]:
# Training Data Paths
training_data = pd.read_csv("analysis_data_sources\CAL10K_with_Spotify_WithFeatures.tab", sep='\t')
training_data

,Unnamed: 0,Song,Genre,Spotify_ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Luther Vandross - If Only For One Night,adult contemporary,2p4WnpR3H16ywshkL2Fp1n,0.498,0.249,9.0,-13.619,0.0,0.0355,0.7550,0.000009,0.0927,0.193,110.636,256240.0,4.0
1,1,Duran Duran - Come Undone,adult contemporary,0yfNXxlyXdmP0ue1iJijx1,0.542,0.793,8.0,-8.203,1.0,0.0530,0.0423,0.000004,0.0814,0.708,173.931,256520.0,4.0
2,2,Tears For Fears - Shout,adult contemporary,2gQaQUhDCNGfBVXTvxAmXQ,0.511,0.942,0.0,-10.407,1.0,0.0429,0.1740,0.000019,0.1390,0.498,97.909,393667.0,4.0
3,3,Marvin Gaye - Sexual healing,adult contemporary,3VZmChrnVW8JK6ano4gSED,0.805,0.601,0.0,-9.700,0.0,0.0579,0.1190,0.000129,0.0581,0.746,94.382,238600.0,4.0
4,4,Jack Johnson - Better Together,adult contemporary,4VywXu6umkIQ2OS0m1I79y,0.868,0.347,5.0,-9.529,1.0,0.0536,0.3080,0.000020,0.1100,0.659,110.059,207680.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24726,24726,Tex Williams - Downtown Poker Club,western swing,0E90tVg9keujSZoNtNMwDu,0.716,0.690,9.0,-11.500,1.0,0.1690,0.7870,0.000246,0.4580,0.853,140.415,168000.0,4.0
24727,24727,The Rockin R's - The Beat,western swing,2wPFy7SAFnt9Nj2TipWcqb,0.606,0.977,1.0,-3.642,1.0,0.0600,0.0206,0.500000,0.1790,0.809,109.410,294027.0,4.0
24728,24728,"Trace Adkins - All Hat, No Cattle",western swing,1mzq1XYonemDdSta7Ghz6E,0.756,0.436,9.0,-8.975,1.0,0.0454,0.5290,0.000000,0.0797,0.586,119.472,183733.0,4.0
24729,24729,Webb Pierce - Even Tho',western swing,3WVs9tUUwfGY1cXSSP2RIj,0.621,0.540,0.0,-8.775,1.0,0.0305,0.7950,0.000140,0.3810,0.892,86.879,148027.0,4.0


In [5]:
training_data = training_data.loc[~(training_data[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature']]==0).all(axis=1)]

In [6]:
X = training_data.drop(columns=['Spotify_ID', 'Song', 'Genre'], axis=1)
y = training_data["Genre"]
print(X.shape, y.shape)

(19041, 14) (19041,)


In [7]:
X.columns

Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [8]:
X.head(10)

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,0.498,0.249,9.0,-13.619,0.0,0.0355,0.7550,0.000009,0.0927,0.193,110.636,256240.0,4.0
1,1,0.542,0.793,8.0,-8.203,1.0,0.0530,0.0423,0.000004,0.0814,0.708,173.931,256520.0,4.0
2,2,0.511,0.942,0.0,-10.407,1.0,0.0429,0.1740,0.000019,0.1390,0.498,97.909,393667.0,4.0
3,3,0.805,0.601,0.0,-9.700,0.0,0.0579,0.1190,0.000129,0.0581,0.746,94.382,238600.0,4.0
4,4,0.868,0.347,5.0,-9.529,1.0,0.0536,0.3080,0.000020,0.1100,0.659,110.059,207680.0,4.0
6,6,0.616,0.597,8.0,-5.328,1.0,0.0415,0.6860,0.000000,0.1040,0.362,165.508,224067.0,4.0
7,7,0.468,0.763,10.0,-6.482,0.0,0.0364,0.1110,0.000000,0.9630,0.324,143.735,251600.0,4.0
8,8,0.573,0.902,6.0,-7.638,0.0,0.0540,0.0180,0.001250,0.0928,0.876,84.412,225280.0,4.0
9,9,0.733,0.566,8.0,-12.060,0.0,0.0390,0.0966,0.000000,0.0740,0.775,141.940,275827.0,4.0
10,10,0.365,0.545,11.0,-9.213,1.0,0.0512,0.4480,0.000000,0.3140,0.459,105.349,235291.0,5.0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
X_train

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
698,698,0.064,0.509,8.0,-12.738,1.0,0.0604,0.000492,0.958000,0.1830,0.049,74.432,246453.0,4.0
5331,5331,0.656,0.497,5.0,-8.956,1.0,0.0728,0.935000,0.303000,0.2420,0.962,198.108,154546.0,4.0
6753,6753,0.583,0.376,6.0,-10.914,1.0,0.0433,0.725000,0.251000,0.0642,0.614,120.713,217373.0,4.0
9634,9634,0.404,0.603,3.0,-14.877,1.0,0.0327,0.493000,0.953000,0.1160,0.985,136.435,99000.0,4.0
8048,8048,0.284,0.696,2.0,-5.661,1.0,0.0398,0.404000,0.000000,0.1150,0.507,198.270,225080.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12386,12386,0.591,0.663,4.0,-7.819,1.0,0.0568,0.001170,0.430000,0.2470,0.415,83.060,216320.0,4.0
17430,17430,0.480,0.453,11.0,-8.192,1.0,0.0291,0.744000,0.000004,0.1080,0.383,146.810,305600.0,3.0
13232,13232,0.431,0.714,10.0,-14.130,0.0,0.0338,0.029900,0.655000,0.3580,0.519,104.065,275973.0,4.0
14419,14419,0.645,0.267,2.0,-9.416,1.0,0.0612,0.693000,0.000022,0.1200,0.393,132.409,225360.0,4.0


# Build a Deep Neural Network

In [11]:
# Create an empty sequential model
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=14))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=147, activation='softmax'))


In [12]:
deep_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1500      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 147)               14847     
Total params: 26,447
Trainable params: 26,447
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
deep_model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
# Use the training data to fit (train) the model
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=360,
    shuffle=True,
    verbose=2
)

Epoch 1/360
447/447 - 0s - loss: 3.6913 - accuracy: 0.2039
Epoch 2/360
447/447 - 0s - loss: 2.6235 - accuracy: 0.3370
Epoch 3/360
447/447 - 0s - loss: 2.2767 - accuracy: 0.3967
Epoch 4/360
447/447 - 0s - loss: 2.0885 - accuracy: 0.4303
Epoch 5/360
447/447 - 0s - loss: 1.9576 - accuracy: 0.4520
Epoch 6/360
447/447 - 0s - loss: 1.8461 - accuracy: 0.4832
Epoch 7/360
447/447 - 0s - loss: 1.7658 - accuracy: 0.4982
Epoch 8/360
447/447 - 0s - loss: 1.6935 - accuracy: 0.5118
Epoch 9/360
447/447 - 0s - loss: 1.6263 - accuracy: 0.5285
Epoch 10/360
447/447 - 0s - loss: 1.5684 - accuracy: 0.5364
Epoch 11/360
447/447 - 0s - loss: 1.5141 - accuracy: 0.5543
Epoch 12/360
447/447 - 0s - loss: 1.4675 - accuracy: 0.5634
Epoch 13/360
447/447 - 0s - loss: 1.4224 - accuracy: 0.5768
Epoch 14/360
447/447 - 0s - loss: 1.3893 - accuracy: 0.5842
Epoch 15/360
447/447 - 0s - loss: 1.3562 - accuracy: 0.5919
Epoch 16/360
447/447 - 0s - loss: 1.3255 - accuracy: 0.6016
Epoch 17/360
447/447 - 0s - loss: 1.2964 - accura

# Save the Trained Model

In [15]:
# Save the model
deep_model.save('genre_trained.h5')

# Evaluate the Model

In [16]:
# Load the model
from tensorflow.keras.models import load_model
deep_model = load_model("genre_trained.h5")

In [17]:
# Evaluate the model using the training data
model_loss, model_accuracy = deep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")


149/149 - 0s - loss: 1.3464 - accuracy: 0.7433
Normal Neural Network - Loss: 1.3464467525482178, Accuracy: 0.743331253528595


In [18]:

encoded_predictions = deep_model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [74]:
encoded_predictions

array([138, 138, 138, 138, 138], dtype=int64)

In [84]:
print(f"Predicted: {list(prediction_labels)}")
print(f"Actual Labels: {list(y_test[0:101])}")

Predicted: ['pop', 'all points west radio', 'early blues', 'jazz', 'rock', 'hard rock', 'new orleans?classic jazz', 'festival', 'western swing', 'rock', 'folk', 'festival', 'symphonic, classical period', 'r&b', 'festival', 'hardly strictly bluegrass festival', 'jazz', 'alternative', 'bebop', 'reggae', 'techno', 'underground hip hop', 'dance', 'instrumental hip hop', 'festival', 'hip hop', 'pop', 'dance', 'classical', 'british folk', 'jazz', 'country', 'country', 'electronic', 'jazz', 'country', 'alternative', 'hip hop', 'pop', 'pop', 'festival', 'festival', 'r&b', 'festival', 'new orleans?classic jazz', 'rock', 'rock', 'dancehall', 'hip hop', 'dancehall', 'trip hop', 'classic pop', 'traditional country', 'disco', 'piano solo, romantic period', 'hard rock', 'nature sounds radio', 'tropical', 'rock', 'country', 'symphonic, romantic period', 'pop', 'alternative', 'salsa', 'festival', 'rock', 'country pop', 'blues', 'reggae', 'soul', 'alternative country', 'jazz', 'contemporary christian',

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [82]:
np.array(X_train.iloc[1:2, :])
# X_train.iloc[1:2, 0] = 0
encoded_predictions = deep_model.predict_classes(X_train_scaled[0:101])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [72]:
X_train_scaled = X_scaler.transform(X_train)

In [79]:
# X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train.iloc[0:101, :])